### Data source 
#### https://www.kaggle.com/c/dogs-vs-cats

you need to divide data into train and val like

- Data
    - train
        - dog
        - cat
    - val
        - dog
        - cat

In [1]:
from tensorflow import keras as keras
import tensorflow as tf

In [2]:
class convBlock(keras.models.Model):
    
    def __init__(self, convNum , filters):
        super().__init__()
        
        self.convNum = convNum
        for i in range(convNum):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(filters, (3,3), 
                                                               activation='relu', padding="same")
            
            
        self.maxPool = keras.layers.MaxPool2D(2  , 2)
    
    
    
    def call(self, inputs):
        
        
        conv2D_0 = vars(self)['conv2D_0']
        x = conv2D_0(inputs)
        for i in range(1,self.convNum):
            
            conv2D_i = vars(self)[f'conv2D_{i}']
            x = conv2D_i(x)
            
            
            
        x = self.maxPool(x)
        
        
        return x
    

In [3]:
class VGG16(keras.models.Model):
    def __init__(self,numberOfClass):
        super().__init__()
        
        self.block1 = convBlock(2 , 64)
        self.block2 = convBlock(2 , 128)
        self.block3 = convBlock(3 , 256) 
        self.block4 = convBlock(3 , 512)
        self.block5 = convBlock(3 , 512) 
        
        self.fcLayer1 = keras.layers.Dense(128 , 
                                          activation="relu")
        self.fcLayer2 = keras.layers.Dense(128 , 
                                          activation="relu")
        
        self.softmaxLayer  = keras.layers.Dense(numberOfClass , 
                                                activation="softmax")
        self.flatten = keras.layers.Flatten()
        
    def call(self, inputs):
        x = self.block1(inputs) #2
        x = self.block2(x) #2
        x = self.block3(x) #3
        x = self.block4(x) #3
        x = self.block5(x) #3
        x = self.flatten(x)
        x = self.fcLayer1(x) #1
        x = self.fcLayer2(x) #1
        x = self.softmaxLayer(x)  #1
           
        return   x


In [4]:
model = VGG16(2)
model.build(input_shape=(0,224,224,3))
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block (convBlock)      multiple                  0         
                                                                 
 conv_block_1 (convBlock)    multiple                  0         
                                                                 
 conv_block_2 (convBlock)    multiple                  0         
                                                                 
 conv_block_3 (convBlock)    multiple                  0         
                                                                 
 conv_block_4 (convBlock)    multiple                  0         
                                                                 
 dense (Dense)               multiple                  3211392   
                                                                 
 dense_1 (Dense)             multiple                  16512 

In [5]:
train_dataGen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
Train_data = train_dataGen.flow_from_directory(
    directory="./Data/train/",
    batch_size=20,
    target_size=(150 ,150),
    class_mode="binary"
)
validation_dataGen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
Validation_data = validation_dataGen.flow_from_directory(
    directory="./Data/val/",
    batch_size=20,
    target_size=(150 ,150),
    class_mode="binary"
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
# create a VGG16 instance with 2 output units for dog vs cat
VGGModel = VGG16(2)
VGGModel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model.

VGGModel.fit(Train_data , validation_data= Validation_data, epochs=1)

100/100 [==============================] - 69s 687ms/step - loss: 0.6938 - accuracy: 0.5095 - val_loss: 0.6924 - val_accuracy: 0.5000
